In [211]:
import os
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.chart import BarChart, Reference
from openpyxl.chart import PieChart, Reference
from openpyxl.chart.marker import DataPoint
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, Border, Side, PatternFill, Font

In [212]:
def get_categories():
    return {
        "kahve": {"STARBUCKS", "DURSUN ARSLAN", "DİNÇERLER", "LAGINA", "KAHVE", "COFFEE", "ARABICA", "CAFFE NERO", "JAX", "DC KAHVE", "COFFEE CURVE"},
        "yemek": {
            "BÖREK","KOFTE", "MARKET", "ASPAVA", "DÖNER", "KÖFTE", "PİLAV", "PİLAVCI", "RESTAURANT", "GIDA",
            "MAMUL", "BURGER", "CAFÉ", "KAFE", "POŞET", "ÇİĞKÖFTECİ", "KASAP", "PASTANE", "MUTFAK",
            "KAHVALTI", "YEMEK", "ETİ", "TATLİ", "KEBAP", "SİMİT", "ÇORBA", "LOKANTA", "TRENDYOL YEMEK", "PILAVCI"
        },
        "giyim": {"LC WAIKIKI", "GİYİM", "KOTON", "ZARA", "MUDO", "DEFACTO", "AYAKKABI", "OPTİK", "KİYAFET", "MONT", "ELBİSE"},
        "online": {"TRENDYOL.COM", "İNTERNET", "ONLINE", "SİPARİŞ", "E-TİCARET", "HEPSİBURADA","HEPSIBURADA", "N11"},
        "ulasim": {"PEGASUS","TURİZM","EGO", "OBILET", "POSFİXİSTANBULKART", "OBILET", "BİLET", "HAVA", "TAŞIMACILIK", "UÇAK", "OTOBÜS", "METRO", "İETT", "TAKSİ", "ULAŞIM"},
        "eglence": {"SİNEMA", "TİYATRO", "KONSER", "OYUN", "CAFE", "MEKAN", "BAR", "AKM", "POPLIST", "GALATA", "EĞLENCE", "KIOSK"},
        "egitim": {"KİTAP", "KİTABEVİ", "DERS", "OKUL", "EĞİTİM", "UNİVERSITE", "KURS", "KIRTASİYE"},
        "abonelik": {"ABONELİK", "SPOTIFY", "NETFLIX", "YOUTUBE", "MEMBERSHIP", "AİDAT", "TİVİBU", "TV+", "EXXEN"}
    }



In [213]:
def clean_data_from_first_valid_date(df):
    """
    Cleans the DataFrame by:
    1. Keeping rows from the first valid date in the first column.
    2. Removing rows in the second column that contain 'ÖDEME-İNTERNET BANKACILIĞI' (case-insensitive, cleaned).
    
    Args:
        df (pd.DataFrame): The input DataFrame.
    
    Returns:
        pd.DataFrame: Cleaned DataFrame.
    """
    # Convert first column to datetime
    mask = pd.to_datetime(df.iloc[:, 0], format="%d/%m/%Y", errors='coerce')
    
    # Get the first valid date index
    first_valid_index = mask.first_valid_index()
    
    # Start from the first valid date
    cleaned_df = df.iloc[first_valid_index:].reset_index(drop=True)
    
    # Clean the second column: strip spaces and convert to uppercase
    second_col = cleaned_df.columns[1]  # Get the name of the second column
    cleaned_df[second_col] = cleaned_df[second_col].astype(str).str.strip().str.upper()
    
    # Remove rows with unwanted value
    cleaned_df = cleaned_df[cleaned_df[second_col] != "ÖDEME-İNTERNET BANKACILIĞI"].reset_index(drop=True)
    
    return cleaned_df


In [214]:
def parse_amount(val):
    val = str(val).replace('.', '').replace(' TL', '').replace(',', '.').strip()
    if val.startswith('+'):
        return float(val[1:])
    else:
        return -float(val)


In [215]:
def kategori_bul(islem, sektor):
    islem_upper = str(islem).upper()
    
    # Check if the category is not already matched
    for kategori, anahtar_kelimeler in categories.items():
        for kelime in anahtar_kelimeler:
            if kelime in islem_upper:
                return kategori.upper()
    
    # If no match in 'İşlemler', check 'Sektör' column for "market"
    if "MARKET" in sektor.upper() or "GIDA" in sektor.upper() :
        return "MARKET"  # Assign "MARKET" if it's found in 'Sektör'
    elif "YEMEK" in sektor.upper():
        return "YEMEK"

    # If no matches found in both 'İşlemler' and 'Sektör', assign "DIGER"
    return "DIGER"

In [216]:
def total_of_categories(df):
    # Sabit kategori listesi (küçük harflerle tanımlı)
    sabit_kategoriler = ["kahve", "yemek", "giyim", "online", "ulasim", "eglence", "egitim", "abonelik", "market", "diger"]

    # Başlangıçta tüm sabit kategorilere 0 ata
    kategori_toplam = {kategori: 0 for kategori in sabit_kategoriler}

    # Kategorileri küçük harfe çevirerek tutarları topla
    for tutar, kategori in zip(df["Tutar(Adj)"], df["Kategori"]):
        kategori_lower = kategori.lower()

        if kategori_lower in kategori_toplam:
            kategori_toplam[kategori_lower] += round(tutar, 2)
        else:
            # Eğer sabit listede yoksa "diger" kategorisine ekle
            kategori_toplam["diger"] += round(tutar, 2)

    return kategori_toplam


In [217]:
def save_df_to_excel(df, excel_path, ay, yıl):
    """
    Saves the DataFrame to an Excel file with a sheet name in "AyYıl" format (e.g., "Ocak2025").
    It also adds styling to the sheet (borders, headers formatting, etc).
    
    Args:
        df (pd.DataFrame): The DataFrame to save.
        excel_path (str): The path where the Excel file will be saved.
        ay (str): The month in Turkish (e.g., "Ocak").
        yıl (str): The year in 4-digit format (e.g., "2025").
    """
    sheet_name = f"{ay}{yıl}"  # Concatenate month and year to create sheet name
    
    # Create a new Excel writer with openpyxl engine (append mode, replace sheet if exists)
    with pd.ExcelWriter(
        excel_path, 
        engine='openpyxl', 
        mode='a', 
        if_sheet_exists='replace'
    ) as writer:
        
        df.to_excel(writer, index=False, sheet_name=sheet_name)

        
        # Access the workbook and the worksheet we just created
        wb = writer.book
        ws = wb[sheet_name]
        
        # Add a table to the sheet
        table = Table(displayName=f"Table_{sheet_name}", ref=f"A1:{chr(65+df.shape[1]-1)}{df.shape[0]+1}")
        ws.add_table(table)
        
        # Apply some styles
        header_fill = PatternFill(start_color="D3D3D3", end_color="D3D3D3", fill_type="solid")
        for cell in ws[1]:
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.fill = header_fill
            cell.border = Border(bottom=Side(style="thin"))
            cell.font = Font(bold=True)
        
        # Apply borders to all cells
        for row in ws.iter_rows(min_row=2, max_row=df.shape[0]+1, min_col=1, max_col=df.shape[1]):
            for cell in row:
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.border = Border(
                    top=Side(style="thin"),
                    left=Side(style="thin"),
                    right=Side(style="thin"),
                    bottom=Side(style="thin")
                )
        
        # Optional: Apply column width adjustments based on the maximum length of data in each column
        for col in ws.columns:
            max_length = 0
            column = col[0].column_letter  # Get the column name
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws.column_dimensions[column].width = adjusted_width
        
        # Add table style (row stripes, etc.)
        table_style = TableStyleInfo(
            name="TableStyleMedium9", showFirstColumn=False,
            showLastColumn=False, showRowStripes=True, showColumnStripes=False
        )
        table.tableStyleInfo = table_style
        
    print(f"Data saved to {excel_path} with sheet name {sheet_name}")


In [218]:


def update_excel_with_new_data(excel_path, df, sheet_name, start_col, start_row, ay, yıl):
    """
    Updates the specified sheet in an Excel file with new data in specific columns.

    Args:
        excel_path (str): The path to the Excel file.
        df (pd.DataFrame): The DataFrame containing the new data.
        sheet_name (str): The sheet name where data will be updated.
        start_col (str): The column letter where the data should start (e.g., 'J').
        start_row (int): The row number where the data should start.
        ay (str): The month name.
        yıl (int): The year.
    """

    kategori_renkleri = {
        "KAHVE": "6F4E37",
        "YEMEK": "FF0000",
        "GIYIM": "0000FF",
        "ONLINE": "00008B",
        "ULASIM": "800080",
        "EGLENCE": "FFA500",
        "EGITIM": "D3D3D3",
        "ABONELIK": "FFC0CB",
        "MARKET": "CCCCFF",
        "DIGER": "FFFFFF"
    }
    
    sheet_name = f"{ay}{yıl}"  # Güncel sheet ismi

    wb = load_workbook(excel_path)
    
    if sheet_name not in wb.sheetnames:
        print(f"Sheet '{sheet_name}' not found, creating a new sheet.")
        wb.create_sheet(sheet_name)
    
    ws = wb[sheet_name]

    # Önce başlıkları J2 ve K2 hücrelerine yaz
    headers = list(df.columns)
    ws[f"{start_col}2"] = headers[0]  # J2
    ws[f"{chr(ord(start_col) + 1)}2"] = headers[1]  # K2

    # Ardından verileri J3, K3... gibi aşağıya yaz
    for row_idx, row in enumerate(df.itertuples(index=False), start=start_row):
        for col_idx, value in enumerate(row):
            col_letter = chr(ord(start_col) + col_idx)
            ws[f"{col_letter}{row_idx}"] = value

    min_row = start_row
    max_row = start_row + len(df) - 1

    labels = Reference(ws, min_col=10, min_row=min_row, max_row=max_row)  # J sütunu
    data = Reference(ws, min_col=11, min_row=min_row - 1, max_row=max_row)  # K sütunu (başlık dahil)

    bar_chart = BarChart()
    bar_chart.title = f"{ay} {yıl} - Kategori Bazlı Harcama"
    bar_chart.y_axis.title = "Tutar (TL)"
    bar_chart.x_axis.title = "Kategori"
    bar_chart.add_data(data, titles_from_data=True)
    bar_chart.set_categories(labels)

    series = bar_chart.series[0]
    series.dPt = []

    for idx, row in enumerate(ws.iter_rows(min_row=min_row, max_row=max_row, min_col=10, max_col=10)):
        kategori = str(row[0].value).upper()
        renk = kategori_renkleri.get(kategori, "CCCCCC")  # Varsayılan gri
        dp = DataPoint(idx=idx)
        dp.graphicalProperties.solidFill = renk
        series.dPt.append(dp)



    ws.add_chart(bar_chart, "M2")


    pie_chart = PieChart()
    pie_chart.title = f"{ay} {yıl} - Harcama Dağılımı (%)"
    pie_chart.add_data(data, titles_from_data=True)
    pie_chart.set_categories(labels)

    pie_series = pie_chart.series[0]
    pie_series.dPt = []

    for idx, row in enumerate(ws.iter_rows(min_row=min_row, max_row=max_row, min_col=10, max_col=10)):
        kategori = str(row[0].value).upper()
        renk = kategori_renkleri.get(kategori, "CCCCCC")
        dp = DataPoint(idx=idx)
        dp.graphicalProperties.solidFill = renk
        pie_series.dPt.append(dp)

    ws.add_chart(pie_chart, "M20")  # Pasta grafiği yerleştir

    wb.save(excel_path)



def get_sheet_data_and_process(excel_path, aylar, yıl):
    for ay in aylar:
        sheet_name = f"{ay}{yıl}"
        
        # Load category keywords from an external function
        categories = get_categories()

        # Try to load the relevant sheet from the Excel file
        try:
            df = pd.read_excel(excel_path, sheet_name=sheet_name)
        except ValueError:
            # If the sheet doesn't exist, handle the error and stop the process
            print(f"Sheet '{sheet_name}' not found, stopping process.")
            return  # Stops the whole process if sheet is not found
        
        # Clean the data: Keep only the rows starting from the first valid date
        df = clean_data_from_first_valid_date(df)
        
        # Check if 'Tutar' column exists
        if 'Tutar' not in df.columns:
            print(f"Column 'Tutar' not found in sheet {sheet_name}, skipping this sheet.")
            continue
        
        # Parse and convert the 'Tutar' column into numeric format with correct sign
        df["Tutar(Adj)"] = df["Tutar"].apply(parse_amount)
        
        # Determine the spending category for each transaction
        df["Kategori"] = df.apply(lambda row: kategori_bul(row["İşlemler"], row["Sektör"]), axis=1)
        
        # Call the function to calculate total amount for each category
        category_totals = total_of_categories(df)
        
        # Convert the result (dictionary-like) into a new DataFrame
        df2 = pd.DataFrame(category_totals.items(), columns=["Kategoriler", "Total Amount"])
        
        # Save the updated DataFrame with parsed amount and category to a new Excel file
        save_df_to_excel(df, "BUDGET_PLANNER.xlsx", ay, yıl)
        
        # Write the total amounts by category to the Excel sheet, starting from column J
        update_excel_with_new_data(
    "BUDGET_PLANNER.xlsx",
    df2,
    sheet_name=sheet_name,
    start_col="J",
    start_row=3,  # Başlığı korumak için 3. satırdan başlıyoruz
    ay=ay,
    yıl=yıl
)


In [219]:
import pandas as pd
import os

def safe_append_to_csv(df2, csv_path, ay, yıl):
    df2["Ay"] = ay
    df2["Yıl"] = yıl

    temp_file = "temp_append.csv"
    df2.to_csv(temp_file, index=False, encoding='utf-8-sig')

    # Şimdi temp dosyasını ana dosyanın sonuna birleştir (manuel, kontrol altında)
    with open(csv_path, 'a', encoding='utf-8-sig') as main_file, open(temp_file, 'r', encoding='utf-8-sig') as temp:
        lines = temp.readlines()
        if os.path.exists(csv_path):  # İlk satır başlık, varsa atla
            lines = lines[1:]
        main_file.writelines(lines)

    os.remove(temp_file)


In [220]:
# Start year
yıl = 2025
aylar = [
        "Ocak", "Subat", "Mart", "Nisan", "Mayis", "Haziran",
        "Temmuz", "Agustos", "Eylul", "Ekim", "Kasim", "Aralik"
]
# Process months until all months are processed

categories = get_categories()
while True:
    get_sheet_data_and_process("EXPENSES.xlsx", aylar, yıl)
    
    # After processing the months, increase the year
    yıl += 1
    print(f"Processing data for year {yıl}...")
    
    # Stop the loop after the first year is processed
    break  # Remove this break to continue processing for every year, or modify as needed


Data saved to BUDGET_PLANNER.xlsx with sheet name Ocak2025
Data saved to BUDGET_PLANNER.xlsx with sheet name Subat2025
Data saved to BUDGET_PLANNER.xlsx with sheet name Mart2025
Data saved to BUDGET_PLANNER.xlsx with sheet name Nisan2025
Sheet 'Mayis2025' not found, stopping process.
Processing data for year 2026...
